In [263]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
import tensorflow as tf
import random

In [264]:
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)

In [265]:
df_orig = pd.read_csv("train.csv")
print(df_orig.shape)
print("Null Counts:")
print(df_orig.isnull().sum()[df_orig.isnull().sum() > 0])

(1460, 81)
Null Counts:
LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [266]:
df = df_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id", "SalePrice"])
print(df.shape)

(1460, 75)


In [267]:
def fill_nulls(df, mean, mode):
    df.fillna(mean, inplace=True)
    df.fillna(mode, inplace=True)

mean = df.mean()
mode = df.mode().iloc[0]
fill_nulls(df, mean, mode)
print(df.isnull().sum()[df.isnull().sum() > 0])

Series([], dtype: int64)


In [268]:
def encode_categorical_features(df, enc):
    # Flatten enc.categories_ which is a list of np.array
    cat_list = np.concatenate(enc.categories_).ravel()
    df[cat_list] = pd.DataFrame(enc.transform(df[categorical_features]).toarray(), index=df.index)
    df = df.drop(columns=categorical_features)
    return df

categorical_features = df.select_dtypes(exclude=[np.number]).columns
onehot = OneHotEncoder(handle_unknown='ignore')
onehot.fit(df[categorical_features])
df = encode_categorical_features(df, onehot)
print(df.shape)

(1460, 203)


In [269]:
def standardize_numerical_features(df):
    numerical_features = df.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

df = standardize_numerical_features(df)

In [270]:
X = df
y = df_orig.SalePrice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(978, 203) (978,)
(482, 203) (482,)


In [278]:
reg = 0.01
model = Sequential()
model.add(Dense(units=90, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
#model.add(Dense(units=90, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=90, activation='relu'))
#model.add(Dropout(0.1))
#model.add(Dense(units=90, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(units=1, activation=None))

model.compile(loss='mean_squared_error',
              optimizer='Adam',
              metrics=['mean_squared_logarithmic_error'])

model.fit(X_train, y_train, epochs=300, batch_size=16)
train_loss, train_msle = model.evaluate(X_train, y_train)
test_loss, test_msle = model.evaluate(X_test, y_test)
print(np.sqrt(train_msle), np.sqrt(test_msle))
# Current best is 0.14
# 0.0034590166626495664 0.27758142531321994
# 0.037311572331001276 0.2505622472808093 - 0.2 Dropout
# 0.12154992763625211 1.035662857959525 - linear

Epoch 1/300
978/978 [==============================] - 6s 6ms/step - loss: 38716832083.2393 - mean_squared_logarithmic_error: 129.0068
Epoch 2/300
978/978 [==============================] - 0s 245us/step - loss: 38713827671.4274 - mean_squared_logarithmic_error: 101.9333
Epoch 3/300
978/978 [==============================] - 0s 248us/step - loss: 38707390618.9611 - mean_squared_logarithmic_error: 85.7753
Epoch 4/300
978/978 [==============================] - 0s 245us/step - loss: 38696307291.0920 - mean_squared_logarithmic_error: 75.0261
Epoch 5/300
978/978 [==============================] - 0s 251us/step - loss: 38680224296.8344 - mean_squared_logarithmic_error: 67.0252
Epoch 6/300
978/978 [==============================] - 0s 275us/step - loss: 38657400647.7219 - mean_squared_logarithmic_error: 60.3805
Epoch 7/300
978/978 [==============================] - 0s 276us/step - loss: 38629206338.4867 - mean_squared_logarithmic_error: 54.9412
Epoch 8/300
978/978 [===========================

978/978 [==============================] - 0s 273us/step - loss: 30219903930.8957 - mean_squared_logarithmic_error: 6.0408
Epoch 62/300
978/978 [==============================] - 0s 246us/step - loss: 30037292302.1350 - mean_squared_logarithmic_error: 5.9149
Epoch 63/300
978/978 [==============================] - 0s 240us/step - loss: 29792993673.6851 - mean_squared_logarithmic_error: 5.7957
Epoch 64/300
978/978 [==============================] - 0s 240us/step - loss: 29577785335.6237 - mean_squared_logarithmic_error: 5.6898
Epoch 65/300
978/978 [==============================] - 0s 242us/step - loss: 29362549142.2495 - mean_squared_logarithmic_error: 5.5207
Epoch 66/300
978/978 [==============================] - 0s 282us/step - loss: 29091125603.9918 - mean_squared_logarithmic_error: 5.4395
Epoch 67/300
978/978 [==============================] - 0s 240us/step - loss: 28872789278.8875 - mean_squared_logarithmic_error: 5.2859
Epoch 68/300
978/978 [==============================] - 0s 28

978/978 [==============================] - 0s 242us/step - loss: 16151474938.2413 - mean_squared_logarithmic_error: 1.8591
Epoch 122/300
978/978 [==============================] - 0s 231us/step - loss: 15957172806.1513 - mean_squared_logarithmic_error: 1.8431
Epoch 123/300
978/978 [==============================] - 0s 233us/step - loss: 15723654120.9652 - mean_squared_logarithmic_error: 1.7991
Epoch 124/300
978/978 [==============================] - 0s 233us/step - loss: 15514051694.9857 - mean_squared_logarithmic_error: 1.7521
Epoch 125/300
978/978 [==============================] - 0s 236us/step - loss: 15332408195.4029 - mean_squared_logarithmic_error: 1.7339
Epoch 126/300
978/978 [==============================] - 0s 234us/step - loss: 15033192628.0900 - mean_squared_logarithmic_error: 1.6997
Epoch 127/300
978/978 [==============================] - 0s 233us/step - loss: 14771307882.2740 - mean_squared_logarithmic_error: 1.6738
Epoch 128/300
978/978 [==============================] 

978/978 [==============================] - 0s 233us/step - loss: 6277468958.1022 - mean_squared_logarithmic_error: 0.6899
Epoch 182/300
978/978 [==============================] - 0s 234us/step - loss: 6318434159.5092 - mean_squared_logarithmic_error: 0.6743
Epoch 183/300
978/978 [==============================] - 0s 231us/step - loss: 6175623276.8916 - mean_squared_logarithmic_error: 0.6422
Epoch 184/300
978/978 [==============================] - 0s 232us/step - loss: 6083430303.6728 - mean_squared_logarithmic_error: 0.6334
Epoch 185/300
978/978 [==============================] - 0s 232us/step - loss: 6082854941.3170 - mean_squared_logarithmic_error: 0.6419
Epoch 186/300
978/978 [==============================] - 0s 236us/step - loss: 5836445439.2147 - mean_squared_logarithmic_error: 0.6241
Epoch 187/300
978/978 [==============================] - 0s 230us/step - loss: 5895641999.4438 - mean_squared_logarithmic_error: 0.6148
Epoch 188/300
978/978 [==============================] - 0s 23

978/978 [==============================] - 0s 237us/step - loss: 3238293158.4785 - mean_squared_logarithmic_error: 0.3009
Epoch 242/300
978/978 [==============================] - 0s 232us/step - loss: 3300545248.7198 - mean_squared_logarithmic_error: 0.3608
Epoch 243/300
978/978 [==============================] - 0s 229us/step - loss: 3208294126.7239 - mean_squared_logarithmic_error: 0.3025
Epoch 244/300
978/978 [==============================] - 0s 233us/step - loss: 3195694251.2556 - mean_squared_logarithmic_error: 0.2846
Epoch 245/300
978/978 [==============================] - 0s 230us/step - loss: 3186698454.6421 - mean_squared_logarithmic_error: 0.3421
Epoch 246/300
978/978 [==============================] - 0s 232us/step - loss: 3194167922.3885 - mean_squared_logarithmic_error: 0.2762
Epoch 247/300
978/978 [==============================] - 0s 228us/step - loss: 3093781900.0409 - mean_squared_logarithmic_error: 0.2755
Epoch 248/300
978/978 [==============================] - 0s 23

482/482 [==============================] - 0s 95us/step
0.41094325271138404 0.8050061034942686


In [272]:
# df_test_orig = pd.read_csv("test.csv")
# df_test = df_test_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id"])
# fill_nulls(df_test, mean, mode)
# df_test = encode_categorical_features(df_test, onehot)
# df_test = standardize_numerical_features(df_test)
# predictions = model.predict(df_test)
# predictions = np.squeeze(predictions)
# print(predictions.shape)
# df_submit = pd.DataFrame({'Id': df_test_orig.Id, 'SalePrice': predictions})
# df_submit.to_csv('submission.csv', index=False)